In [ ]:
sqlite_db_path = "sqlite:///optuna_study.db"
study_name = "optimize_curvature_lr"
study = optuna.create_study(study_name=study_name, storage=sqlite_db_path, direction='minimize', load_if_exists=True)
study.optimize(objective, n_trials=2)
# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)
print("Best accuracy: ", study.best_value)